In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
import cv2


def load_files_from_dir(path):
    data = {}
    list_files = list(path.glob('*.npz'))
    for p in tqdm(list_files):
        try:
            d = np.load(p)
        except:
            continue
        for k, v in d.items():
            data.setdefault(k, []).extend( v)
    if not data:
        try:
            data = np.load(path.with_suffix('.npz'))
        except:
            raise FileNotFoundError(f'No files found in {path}')
    indices = np.argsort(data['time_ns'])
    data = {k:np.stack(v)[indices] for k, v in data.items()}
    return data


path_to_files = Path('/mnt/e/Downloads/meas/calibration')

left = load_files_from_dir(path_to_files / 'pan')
right = load_files_from_dir(path_to_files / 'mono')
list(right.keys())

In [ ]:
distance_between_centers_of_cameras = 0.1  # meters   ?????????????????????????????????????????
distance_from_ground = 50 # meters
focal_length = 9.8 * 1e-3 # meters
size_of_pixel = 17 * 1e-6 # meters
image_width = 336 # pixels
image_height = 256 # pixels

delta_distance_from_ground = 3 # meters

In [ ]:
fov_width_rad = 2 * np.arctan(image_width * size_of_pixel / (2 * focal_length))
fov_width_degree = fov_width_rad * 180 / np.pi
fov_width_meters = 2 * distance_from_ground * np.tan(fov_width_rad / 2)
fov_height_rad = 2 * np.arctan(image_height * size_of_pixel / (2 * focal_length))
fov_height_degree = fov_height_rad * 180 / np.pi
fov_height_meters = 2 * distance_from_ground * np.tan(fov_height_rad / 2)
print(f"FOV width: {fov_width_degree:.1f}deg, {fov_width_meters:.1f}m")
print(f"FOV height: {fov_height_degree:.1f}deg, {fov_height_meters:.1f}m")


In [ ]:
fov_single_pixel_width_meters = 2 * distance_from_ground * size_of_pixel / focal_length
fov_single_pixel_height_meters = 2 * distance_from_ground * size_of_pixel / focal_length
print(f"FOV single pixel width: {fov_single_pixel_width_meters:.2g}m")
print(f"FOV single pixel height: {fov_single_pixel_height_meters:.2g}m")

min_fov_change = 2 * (distance_from_ground-delta_distance_from_ground) * size_of_pixel / focal_length
max_fov_change = 2 * (distance_from_ground+delta_distance_from_ground) * size_of_pixel / focal_length
print(f"Min FOV change: {min_fov_change:.2g}m")
print(f"Max FOV change: {max_fov_change:.2g}m")

In [ ]:
distance_between_cameras_in_pixels = distance_between_centers_of_cameras / fov_single_pixel_width_meters
print(f"Distance between cameras in pixels: {distance_between_cameras_in_pixels:.2g} pixels")

In [ ]:
# plot the first frame side-by-side
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(left['frames'][0])
ax[1].imshow(right['frames'][0])
ax[0].set_title('Left camera')
ax[1].set_title('Right camera')
plt.show()

In [ ]:
# Use cv2 to find the homography between the frames

def find_homography(left, right):
    left = cv2.normalize(left, left, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    right = cv2.normalize(right, right, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(left, None)
    kp2, des2 = orb.detectAndCompute(right, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M, mask
# find_homography(left['frames'][0], right['frames'][0])[0]

In [ ]:
# def warp_image(img, M):
#     return cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))


# N = 10
# array_distances = np.linspace(-5, 5, N)
# M_permutations = np.eye(3)[None, None]
# M_permutations = M_permutations.repeat(N, axis=0).repeat(N, axis=1)
# M_permutations[:, :, 0, 2] = array_distances[:, None]
# M_permutations[:, :, 1, 2] = array_distances[None, :]

# with tqdm(total=len(left['frames']) * N**2) as pbar:
#     best_loss, best_M = float('inf'), None
#     for i,j in np.ndindex(N, N):
#         loss = 0
#         M = M_permutations[i, j]
#         for idx, frame in enumerate(left['frames']):
#             estimated_right = warp_image(left['frames'][0], M)
#             loss += np.mean(np.abs((estimated_right - right['frames'][idx])))
#             pbar.update(1)
#         loss /= len(left['frames'])
#         if loss < best_loss:
#             best_loss = loss
#             best_M = M
#         pbar.set_postfix_str(f'loss: {loss:.2f}, best_loss: {best_loss:.2f}')
#         pbar.set_description_str(f"i: {i}, j: {j}")


In [ ]:
from ctypes import c_float
best_loss = mp.Value(typecode_or_type=c_float)
best_loss.value = float('inf')

def warp_image(img, M):
    return cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))

def _calc_loss(M, left, right):
    loss = 0
    for idx in range(len(left['frames'])):
        estimated_right = warp_image(left['frames'][idx], M)
        loss += np.mean(np.abs((estimated_right - right['frames'][idx])))
    loss /= len(left['frames'])
    best_loss.value = min(loss, best_loss.value)
    print(f"loss: {loss:.2f}, best_loss: {best_loss.value:.2f}")
    return loss


N = 100
N_PIXELS = 2
array_distances = np.linspace(-N_PIXELS, N_PIXELS, N)
M_permutations = np.eye(3)[None, None]
M_permutations = M_permutations.repeat(N, axis=0).repeat(N, axis=1)
M_permutations[:, :, 0, 2] = array_distances[:, None]
M_permutations[:, :, 1, 2] = array_distances[None, :]
M_permutations = M_permutations.reshape(-1, 3, 3)

func = partial(_calc_loss, left=left.copy(), right=right.copy())


with mp.Pool(mp.cpu_count()) as pool:
    ret_val = list(tqdm(pool.imap(func, M_permutations, chunksize=2**4), total=len(M_permutations)))

print(ret_val.min())
print(M_permutations[np.argmin(ret_val)])
